In [1]:
  
#-*- coding: utf-8 -*-

from __future__ import unicode_literals, print_function, division
from comet_ml import Experiment
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import matplotlib.pyplot as plt


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


SOS_token = 0
EOS_token = 1

# hyperparams = {
# 	"rnn_size": 128,  # assuming encoder and decoder use the same rnn_size
# 	"embedding_size": 128,
# 	"num_epochs": 20,
# 	"batch_size": 1,
# 	"learning_rate": 0.001,
# 	"max_sequence_length": 20,
# 	}

experiment = Experiment(api_key="pTKIV6STi3dM8kJmovb0xt0Ed",
                        project_name="lang2ltl", workspace="suchzheng2")

# experiment.log_parameters(hyperparams)

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1


def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )



def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    return s



def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open(".\input_500_originaltext.csv").\
        read().strip().split('\n')
    pairs = [[normalizeString(s) for s in l.split(',')[1:]] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs


MAX_LENGTH = 20


def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('lang', 'ltl', False)
#print(random.choice(pairs))


class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)




class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)




def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    print(pair)
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)




teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    output_tokens = []
    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input
            output_tokens.append(decoder_input)
            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()
    
    if loss.item() / target_length > 10:
        predict = [output_lang.index2word[id] for id in output_tokens]
        target = [output_lang.index2word[id] for id in target_tensor]
        print("="," ",join(predict))
        print("<"," ",join(target))


    return loss.item() / target_length
    #return loss


import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))




def trainIters(encoder, decoder,  training_pairs, print_every=1000, plot_every=100, learning_rate=0.01, val_every = 100):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    criterion = nn.NLLLoss()
    
    n_iters = len(training_pairs)
    curr_loss = []
    spikes = []
    with experiment.train():
        
        for iter in range(1, n_iters + 1):
            training_pair = training_pairs[iter - 1]
            input_tensor = training_pair[0]
            target_tensor = training_pair[1]

            loss = train(input_tensor, target_tensor, encoder,
                        decoder, encoder_optimizer, decoder_optimizer, criterion)

            experiment.log_metric("train_loss",loss)
            print_loss_total += loss
            plot_loss_total += loss

            if iter % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                            iter, iter / n_iters * 100, print_loss_avg))

            if iter % plot_every == 0:
                plot_loss_avg = plot_loss_total / plot_every
                plot_losses.append(plot_loss_avg)
                plot_loss_total = 0

#             if iter % val_every == 0:
#                 evaluate_model(encoder1, attn_decoder1,validation_pairs)

            # curr_loss += loss 

        # curr_loss.backward()
        
        # encoder_optimizer.step()
        # decoder_optimizer.step()
    #showPlot(plot_losses)


######################################################################
# Plotting results
# ----------------
#
# Plotting is done with matplotlib, using the array of loss values
# ``plot_losses`` saved while training.
#

import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)


######################################################################
# Evaluation
# ==========
#
# Evaluation is mostly the same as training, but there are no targets so
# we simply feed the decoder's predictions back to itself for each step.
# Every time it predicts a word we add it to the output string, and if it
# predicts the EOS token we stop there. We also store the decoder's
# attention outputs for display later.
#

def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
    
        decoder_attentions = torch.zeros(max_length, max_length)
        decoder_outputs = torch.zeros(max_length,output_lang.n_words)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            decoder_outputs[di] = decoder_output
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1],decoder_outputs


######################################################################
# We can evaluate random sentences from the training set and print out the
# input, target, and output to make some subjective quality judgements:
#

def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

def evaluate_model(encoder, decoder,evaluate_pairs):
    criterion = nn.NLLLoss()
    cur_loss = 0
    num_target = 0
    with experiment.validate(), torch.no_grad():
        for pair in evaluate_pairs:
            print('>', pair[0])
            print('=', pair[1])
            output_words, attentions, decoder_outputs = evaluate(encoder, decoder, pair[0])
            output_sentence = ' '.join(output_words)
            print('<', output_sentence)
            print('')

            target_tensor = tensorFromSentence(output_lang, pair[1])
            target_tensor = target_tensor.to(device)
            sentence_loss = 0
            for di in range(len(target_tensor)):
                loss = criterion(decoder_outputs[di].unsqueeze(0).to(device),target_tensor[di].to(device))
                #experiment.log_metric("loss ", loss)
                sentence_loss += loss
                cur_loss += loss
                
            
            #perplexity = torch.exp(sentence_loss/len(target_tensor))
            #experiment.log_metric("validation perplexity ", perplexity)
            num_target += len(target_tensor)
        
        #print("validation curr loss: ",cur_loss)
        print("validation loss: ",cur_loss/num_target)
        experiment.log_metric("loss: ",cur_loss/num_target)
        return cur_loss/num_target
        #print("validation perplexity: ",perplexity)




COMET INFO: Experiment is live on comet.ml https://www.comet.ml/suchzheng2/lang2ltl/2b217af54df84a54966586d0d46ac8ca



Reading lines...
Read 556 sentence pairs
Counting words...
Counted words:
lang 45
ltl 35


# Part 1: Add name entiry recognition

In [2]:
#load trained spacy model on location recognition. 

import spacy

location_sniffer_path = "..\\location-sniffer\\trained_model"
location_sniffer = spacy.load(location_sniffer_path)

In [3]:
#test the location sniffer
text = "Go to starbucks and find the bookstore"
doc = location_sniffer(text)
print(text)
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Go to starbucks and find the bookstore
Entities [('starbucks', 'LOC'), ('bookstore', 'LOC')]


In [4]:
loc2var_map = []
var2loc_map = []
location_var= '[loc#]'

In [5]:
def findAndReplaceLocations(pair):
    sent1 = pair[0]
    sent2 = pair[1]
    
    charAnd = '&'
    
    pos1= sent1.find(charAnd)
    if pos1!= -1:
        sent1 = sent1.replace(charAnd,"")
    locs1 = location_sniffer(sent1)
    if pos1!= -1:
        sent1 = sent1[:pos1]+charAnd+sent1[pos1:]
#     print(pair[0])
#     print("Entities", [(ent.text, ent.label_) for ent in locs1.ents])
    
    pos2= sent2.find(charAnd)
    if pos2!= -1:
        sent2 = sent2.replace(charAnd,"")
    locs2 = location_sniffer(sent2)
    if pos2!= -1:
        sent2 = sent2[:pos2]+charAnd+sent2[pos2:]
#     print(pair[1])
#     print("Entities", [(ent.text, ent.label_) for ent in locs2.ents])

    sent_loc2var= {}
    sent_var2loc= {}
    #Todo: randomize location var
    numLoc = len(locs1.ents)
    for idx, ent in enumerate(locs1.ents):
        varName = location_var.replace("#",str(idx))
        location = ent.text
        sent_loc2var[location] = varName
        sent_var2loc[varName] = location
        
        sent1 = sent1.replace(location,varName) 
        sent2 = sent2.replace(location,varName)
    

    loc2var_map.append(sent_loc2var)
    var2loc_map.append(sent_var2loc)
    
#     print(pair[0])
#     print(sent1)
#     print(pair[1])
#     print(sent2)
    
    return(sent1,sent2)
    

In [6]:
def readLangs2(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open(".\input_500_originaltext.csv").read().strip().split('\n')
    pairs = [[s for s in l.split(',')[1:]] for l in lines[1:]]
    pairs = [[s for s in p] for p in pairs]
    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)
    return input_lang, output_lang, pairs

def prepareData2(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs2(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    print("Counting words...")
    for idx, pair in enumerate(pairs):
        
        pair = findAndReplaceLocations(pair)
        print(pair)
        pair = [normalizeString(s) for s in pair]
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
        pairs[idx] = pair
    
#     print("Counted words:")
    
#     print(input_lang.name, input_lang.n_words)
#     print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData2('lang', 'ltl', False)

Reading lines...
Read 555 sentence pairs
Counting words...
('Find the [loc0]', 'F ( [loc0] )')
('Go to [loc0]', 'F ( [loc0] )')
('Find the [loc0]', 'F ( [loc0] )')
('Go to [loc0]', 'F ( [loc0] )')
('Go to the [loc0]', 'F ( [loc0] )')
('Find the [loc0]', 'F ( [loc0] )')
('Go to the [loc0]', 'F ( [loc0] )')
('Find where is the [loc0]', 'F ( [loc0] )')
('Go to the [loc0] and [loc1]', 'F ( [loc0] and [loc1] )')
('Find the [loc0] and [loc1]', 'F ( [loc0] and [loc1] )')
('Find the [loc0]', 'F ( [loc0] )')
('Discover where is the [loc0]', 'F ( [loc0] )')
('Find the [loc0]', 'F ( [loc0] )')
('Go to the [loc0]', 'F ( [loc0] )')
('Find the [loc0] ', 'F ( [loc0] )')
('Go to the [loc0]', 'F ( [loc0] )')
('Find the [loc0]', 'F ( [loc0] )')
('Go to the [loc0]', 'F ( [loc0] )')
('Go to the [loc0]', 'F ( [loc0] ) ')
('Find the [loc0]', 'F ( [loc0] ) ')
('Go to the [loc0]', 'F ( [loc0] )')
('Find the [loc0]', 'F ( [loc0] )')
('Find the [loc0]', 'F ( [loc0] )')
('Go to find the [loc0]', 'F ( [loc0] )')


('Find the [loc0] and then find the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Find the [loc0] and then find the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Find the [loc0] and then find the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Find the [loc0] and then find the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Find the [loc0] and then find the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Find the [loc0] and then find the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Find the [loc0] and then find the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Find the [loc0] and then find the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Find the [loc0] and then find the [loc1] and [loc2]', 'F ( [loc0] & F ( [loc1] and [loc2] ) )')
('Find the [loc0] and then find the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Find the [loc0] and then find the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Find the [loc0] and then find the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Find the [loc0] and then find the [loc1]', 'F ( [loc0] & F ( [lo

('Go to the [loc0] and then go to the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Go to the [loc0] and then go to the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Go to the [loc0] and then go to the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Go to the [loc0] and then go to the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Go to the [loc0] and then go to the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Go to the [loc0] and then go to the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Go to the [loc0] and then go to the Kabob and [loc1]', 'F ( [loc0] & F ( Kabob and [loc1] ) )')
('Go to the [loc0] and then go to the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Go to the [loc0] and then go to the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Go to the [loc0] and then go to the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Go to the [loc0] and then go to the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Go to the [loc0] and then go to the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Go to the [loc0] and then go to the [loc1]

('Stay away from [loc0] and find the [loc1]', 'G ( ! [loc0] ) & F ( [loc1] )')
('Stay away from [loc0] and find the [loc1]', 'G ( ! [loc0] ) & F ( [loc1] )')
('Stay away from [loc0] and find the [loc1]', 'G ( ! [loc0] ) & F ( [loc1] )')
('Stay away from [loc0] and find the [loc1]', 'G ( ! [loc0] ) & F ( [loc1] )')
('Stay away from [loc0] and find the [loc1]', 'G ( ! [loc0] ) & F ( [loc1] )')
('Stay away from [loc0] and find the [loc1]', 'G ( ! [loc0] ) & F ( [loc1] )')
('Stay away from [loc0] and find the [loc1]', 'G ( ! [loc0] ) & F ( [loc1] )')
('Stay away from [loc0] and find the [loc1]', 'G ( ! [loc0] ) & F ( [loc1] )')
('Stay away from [loc0] and find the [loc1] and [loc2]', 'G ( ! [loc0] ) & F ( [loc1] and [loc2] )')
('Stay away from [loc0] and find the [loc1]', 'G ( ! [loc0] ) & F ( [loc1] )')
('Stay away from [loc0] and find the [loc1]', 'G ( ! [loc0] ) & F ( [loc1] )')
('Stay away from [loc0] and find the [loc1]', 'G ( ! [loc0] ) & F ( [loc1] )')
('Stay away from [loc0] and fi

('Do not enter [loc0] first find the [loc1] and then find the [loc2]', 'G ( ! [loc0] ) & F ( [loc1]  F ( [loc2] ) )')
('Do not enter [loc0] first find the [loc1] and then find the [loc2]', 'G ( ! [loc0] ) & F ( [loc1]  F ( [loc2] ) )')
('Do not enter [loc0] first find the [loc1] and then find the [loc2]', 'G ( ! [loc0] ) & F ( [loc1]  F ( [loc2] ) )')
('Do not enter [loc0] first find the [loc1] and then find the [loc2] and [loc3]', 'G ( ! [loc0] ) & F ( [loc1]  F ( [loc2] and [loc3] ) )')
('Do not enter [loc0] first find the [loc1] and then find the [loc2]', 'G ( ! [loc0] ) & F ( [loc1]  F ( [loc2] ) )')
('Do not enter [loc0] first find the [loc1] and then find the [loc2]', 'G ( ! [loc0] ) & F ( [loc1]  F ( [loc2] ) )')
('Do not enter [loc0] first find the [loc1] and then find the [loc2]', 'G ( ! [loc0] ) & F ( [loc1]  F ( [loc2] ) )')
('Do not enter [loc0] first find the [loc1] and then find the [loc2] and [loc3]', 'G ( ! [loc0] ) & F ( [loc1]  F ( [loc2] and [loc3] ) )')
('Do not ent

('Do not enter [loc0] first find the [loc1] and then find the [loc2]', 'G ( ! [loc0] ) & F ( [loc1]  F ( [loc2] ) )')
('Do not enter [loc0] first find the [loc1] and then find the [loc2] and [loc3]', 'G ( ! [loc0] ) & F ( [loc1]  F ( [loc2] and [loc3] ) )')
('Do not enter [loc0] first find the [loc1] and then find the [loc2]', 'G ( ! [loc0] ) & F ( [loc1]  F ( [loc2] ) )')
('Do not enter [loc0] first find the [loc1] and then find the [loc2]', 'G ( ! [loc0] ) & F ( [loc1]  F ( [loc2] ) )')
('Do not enter [loc0] first find the [loc1] and then find the [loc2]', 'G ( ! [loc0] ) & F ( [loc1]  F ( [loc2] ) )')
('Do not enter [loc0] first find the [loc1] and then find the [loc2]', 'G ( ! [loc0] ) & F ( [loc1]  F ( [loc2] ) )')
('Do not enter [loc0] first find the [loc1] and then find the [loc2]', 'G ( ! [loc0] ) & F ( [loc1]  F ( [loc2] ) )')
('Do not enter [loc0] first find the [loc1] and then find the [loc2]', 'G ( ! [loc0] ) & F ( [loc1]  F ( [loc2] ) )')
('Do not enter [loc0] first find t

In [39]:
def findAndReplaceLocationsOneSent(input_sentence):
       
    locs1 = location_sniffer(input_sentence)
   
    sent_loc2var= {}
    sent_var2loc= {}
#     Todo: randomize location var
    numLoc = len(locs1.ents)
    for idx, ent in enumerate(locs1.ents):
        varName = location_var.replace("#",str(idx))
        location = ent.text
        sent_loc2var[location] = varName
        sent_var2loc[varName] = location
        
        input_sentence = input_sentence.replace(location,varName) 
      
    return input_sentence,sent_loc2var,sent_var2loc
#     numLoc = len(locs1)
#     for idx, ent in enumerate(locs1):
#         varName = location_var.replace("#",str(idx))
#         location = ent[0]
#         sent_loc2var[location] = varName
#         sent_var2loc[varName] = location
        
#         input_sentence = input_sentence.replace(location,varName) 
      
#     return input_sentence,sent_loc2var,sent_var2loc


def evaluateExampleWithName(input_sentence):
    sent, loc2var, var2loc = findAndReplaceLocationsOneSent(input_sentence)
    print('input =         ', input_sentence)
    print('masked input = ',sent)
    output_words = evaluate(
        encoder2, attn_decoder2, sent)
   
    #print(output_words)
    output = ' '.join(output_words[0])
    print('model output =', output)
    for (var,loc) in var2loc.items():
        output = output.replace(var,loc)
    print('mapped output =', output)
    
    #showAttention(input_sentence, output_words, attentions)

In [38]:
input_lang2, output_lang2, pairs2 = prepareData2('lang', 'ltl', False)

hidden_size2 = 256
num_epoch2 = 5
encoder2 = EncoderRNN(input_lang2.n_words, hidden_size2).to(device)
attn_decoder2 = AttnDecoderRNN(hidden_size2, output_lang2.n_words, dropout_p=0.1).to(device)
train_split = 0.8
valid_split = 0.1

#random.shuffle(pairs)

temp = list(zip(pairs2,var2loc_map)) 
random.shuffle(temp) 
pairs2, res2= zip(*temp) 

num_train_pairs2 = int(len(pairs2)*0.8)
num_valid_pairs2 = int(len(pairs2)*0.1)
num_test_pairs2 = int(len(pairs2) - num_train_pairs2 - num_valid_pairs2)

training_pairs2 = pairs2[:num_train_pairs2]
validation_pairs2 = pairs2[num_train_pairs2:num_train_pairs2+num_valid_pairs2]
testing_pairs2 = pairs2[num_train_pairs2+num_valid_pairs2:]

training_pairs2 = [tensorsFromPair(pair)
                      for pair in pairs2[:num_train_pairs2]]
for i in range(num_epoch2):
    print("=============== Epoch: %d =================="%i)
    random.shuffle(training_pairs2)
    trainIters(encoder2, attn_decoder2, training_pairs2, print_every=100,val_every = 10)
    evaluate_model(encoder2, attn_decoder2,validation_pairs2)

Reading lines...
Read 555 sentence pairs
Counting words...
('Find the [loc0]', 'F ( [loc0] )')
('Go to [loc0]', 'F ( [loc0] )')
('Find the [loc0]', 'F ( [loc0] )')
('Go to [loc0]', 'F ( [loc0] )')
('Go to the [loc0]', 'F ( [loc0] )')
('Find the [loc0]', 'F ( [loc0] )')
('Go to the [loc0]', 'F ( [loc0] )')
('Find where is the [loc0]', 'F ( [loc0] )')
('Go to the [loc0] and [loc1]', 'F ( [loc0] and [loc1] )')
('Find the [loc0] and [loc1]', 'F ( [loc0] and [loc1] )')
('Find the [loc0]', 'F ( [loc0] )')
('Discover where is the [loc0]', 'F ( [loc0] )')
('Find the [loc0]', 'F ( [loc0] )')
('Go to the [loc0]', 'F ( [loc0] )')
('Find the [loc0] ', 'F ( [loc0] )')
('Go to the [loc0]', 'F ( [loc0] )')
('Find the [loc0]', 'F ( [loc0] )')
('Go to the [loc0]', 'F ( [loc0] )')
('Go to the [loc0]', 'F ( [loc0] ) ')
('Find the [loc0]', 'F ( [loc0] ) ')
('Go to the [loc0]', 'F ( [loc0] )')
('Find the [loc0]', 'F ( [loc0] )')
('Find the [loc0]', 'F ( [loc0] )')
('Go to find the [loc0]', 'F ( [loc0] )')


('Find the [loc0] and then find the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Find the [loc0] and then find the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Find the [loc0] and then find the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Find the [loc0] and then find the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Find the [loc0] and then find the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Find the [loc0] and then find the [loc1] and [loc2]', 'F ( [loc0] & F ( [loc1] and [loc2] ) )')
('Find the [loc0] and then find the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Find the [loc0] and then find the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Find the [loc0] and then find the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Find the [loc0] and then find the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Find the [loc0] and then find the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Find the [loc0] and then find the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Find the [loc0] and then find the [loc1]', 'F ( [loc0] & F ( [lo

('Go to the [loc0] and then go to the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Go to the [loc0] and then go to the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Go to the [loc0] and then go to the [loc1] and [loc2]', 'F ( [loc0] & F ( [loc1] and [loc2] ) )')
('Go to the [loc0] and then go to the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Go to the [loc0] and then go to the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Go to the [loc0] and then go to the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Go to the [loc0] and then go to the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Go to the [loc0] and then go to the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Go to the [loc0] and then go to the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Go to the [loc0] and then go to the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Go to the [loc0] and then go to the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Go to the [loc0] and then go to the [loc1]', 'F ( [loc0] & F ( [loc1] ) )')
('Go to the [loc0] and then go to the [loc

('Stay away from [loc0] and find the [loc1]', 'G ( ! [loc0] ) & F ( [loc1] )')
('Stay away from [loc0] and find the [loc1]', 'G ( ! [loc0] ) & F ( [loc1] )')
('Stay away from [loc0] and find the [loc1]', 'G ( ! [loc0] ) & F ( [loc1] )')
('Stay away from [loc0] and find the [loc1]', 'G ( ! [loc0] ) & F ( [loc1] )')
('Stay away from [loc0] and find the [loc1]', 'G ( ! [loc0] ) & F ( [loc1] )')
('Stay on the [loc0] first find the [loc1] and then find the [loc2]', 'G ( [loc0] ) & F ( [loc1]  F ( [loc2] ) )')
('Stay on the [loc0] first find the [loc1] and then find the [loc2]', 'G ( [loc0] ) & F ( [loc1]  F ( [loc2] ) )')
('Stay on the [loc0] first find the [loc1] and then find the [loc2]', 'G ( [loc0] ) & F ( [loc1]  F ( [loc2] ) )')
('Stay on the [loc0] first find the [loc1] and then find the [loc2] and [loc3]', 'G ( [loc0] ) & F ( [loc1]  F ( [loc2] and [loc3] ) )')
('Stay on the [loc0] first find the [loc1] and then find the [loc2]', 'G ( [loc0] ) & F ( [loc1]  F ( [loc2] ) )')
('Stay o

('Do not enter [loc0] first find the [loc1] and then find the [loc2]', 'G ( ! [loc0] ) & F ( [loc1]  F ( [loc2] ) )')
('Do not enter [loc0] first find the [loc1] and then find the [loc2]', 'G ( ! [loc0] ) & F ( [loc1]  F ( [loc2] ) )')
('Do not enter [loc0] first find the [loc1] and then find the Kabob and [loc2]', 'G ( ! [loc0] ) & F ( [loc1]  F ( Kabob and [loc2] ) )')
('Do not enter [loc0] first find the [loc1] and then find the [loc2]', 'G ( ! [loc0] ) & F ( [loc1]  F ( [loc2] ) )')
('Do not enter [loc0] first find the [loc1] and then find the [loc2]', 'G ( ! [loc0] ) & F ( [loc1]  F ( [loc2] ) )')
('Do not enter [loc0] first find the [loc1] and then find the [loc2]', 'G ( ! [loc0] ) & F ( [loc1]  F ( [loc2] ) )')
('Do not enter [loc0] first find the [loc1] and then find the [loc2]', 'G ( ! [loc0] ) & F ( [loc1]  F ( [loc2] ) )')
('Do not enter [loc0] first find the [loc1] and then find the [loc2] and [loc3]', 'G ( ! [loc0] ) & F ( [loc1]  F ( [loc2] and [loc3] ) )')
('Do not enter

...

0m 2s (- 0m 9s) (100 22%) 1.2390
0m 5s (- 0m 6s) (200 45%) 1.0322
0m 8s (- 0m 4s) (300 67%) 0.8348
0m 11s (- 0m 1s) (400 90%) 0.7325
> stay on the [loc0] and find the [loc1]
= g ( [loc0] ) & f ( [loc1] )
< f ( [loc0] & f ( [loc1] ) ) <EOS>

> do not enter [loc0] and find the [loc1]
= g ( ! [loc0] ) & f ( [loc1] )
< g ( ! [loc0] ) & f ( [loc1] ) ) <EOS>

> stay away from [loc0] and find the [loc1]
= g ( ! [loc0] ) & f ( [loc1] )
< g ( ! [loc0] ) & f ( [loc1] ) ) <EOS>

> go to the [loc0] and then go to the [loc1]
= f ( [loc0] & f ( [loc1] ) )
< f ( [loc0] & f ( [loc1] ) ) <EOS>

> go to the [loc0] and then go to the [loc1]
= f ( [loc0] & f ( [loc1] ) )
< f ( [loc0] & f ( [loc1] ) ) <EOS>

> go to the [loc0] and [loc1] and then go to the [loc2]
= f ( [loc0] and [loc1] & f ( [loc2] ) )
< f ( [loc0] & f ( [loc1] ) ) <EOS>

> do not enter [loc0] first find the [loc1] and then find the [loc2]
= g ( ! [loc0] ) & f ( [loc1]  f ( [loc2] ) )
< g ( ! [loc0] ) & f ( [loc1]  f ( [loc2] ) ) <EOS>

>

< g ( ! [loc0] ) & f ( [loc1] ) <EOS>

> stay on the [loc0] first find the [loc1] and then find the [loc2]
= g ( [loc0] ) & f ( [loc1]  f ( [loc2] ) )
< g ( [loc0] ) & f ( [loc1]  f ( [loc2] ) ) <EOS>

> go to the [loc0] and then go to the [loc1]
= f ( [loc0] & f ( [loc1] ) )
< f ( [loc0] & f ( [loc1] ) ) <EOS>

> find the [loc0]
= f ( [loc0] )
< f ( [loc0] ) <EOS>

> find the [loc0] and then find the [loc1]
= f ( [loc0] & f ( [loc1] ) )
< f ( [loc0] & f ( [loc1] ) ) <EOS>

> stay on the [loc0] first find the cvs and then find the [loc1] and [loc2]
= g ( [loc0] ) & f ( cvs  f ( [loc1] and [loc2] ) )
< g ( [loc0] ) & f ( [loc1]  f ( [loc2] and [loc3] ) ) <EOS>

> stay on the [loc0] first find the [loc1] and then find the [loc2]
= g ( [loc0] ) & f ( [loc1]  f ( [loc2] ) )
< g ( [loc0] ) & f ( [loc1]  f ( [loc2] ) ) <EOS>

> go to the [loc0] and then go to the [loc1]
= f ( [loc0] & f ( [loc1] ) )
< f ( [loc0] & f ( [loc1] ) ) <EOS>

> do not enter [loc0] first find the [loc1] and then fin

> do not enter [loc0] first find the [loc1] and [loc2] and then find the [loc3]
= g ( ! [loc0] ) & f ( [loc1] and [loc2]  f ( [loc3] ) )
< g ( ! [loc0] ) & f ( [loc1] and [loc2]  f ( [loc2] ) ) <EOS>

> do not enter [loc0] first find the [loc1] and then find the [loc2]
= g ( ! [loc0] ) & f ( [loc1]  f ( [loc2] ) )
< g ( ! [loc0] ) & f ( [loc1]  f ( [loc2] ) ) <EOS>

> do not enter [loc0] first find the [loc1] and then find the [loc2]
= g ( ! [loc0] ) & f ( [loc1]  f ( [loc2] ) )
< g ( ! [loc0] ) & f ( [loc1]  f ( [loc2] ) ) <EOS>

> do not enter [loc0] first find the [loc1] and then find the [loc2] and [loc3]
= g ( ! [loc0] ) & f ( [loc1]  f ( [loc2] and [loc3] ) )
< g ( ! [loc0] ) & f ( [loc1]  f ( [loc2] ) ) <EOS>

> go to the [loc0] and then go to the [loc1]
= f ( [loc0] & f ( [loc1] ) )
< f ( [loc0] & f ( [loc1] ) ) <EOS>

> stay on the [loc0] first find the [loc1] and then find the [loc2]
= g ( [loc0] ) & f ( [loc1]  f ( [loc2] ) )
< g ( [loc0] ) & f ( [loc1]  f ( [loc2] ) ) <EOS>

> find the [loc0]
= f ( [loc0] )
< f ( [loc0] ) <EOS>

> stay away from [loc0] and find the [loc1]
= g ( ! [loc0] ) & f ( [loc1] )
< g ( ! [loc0] ) & f ( [loc1] ) <EOS>

> do not leave the [loc0] and find the [loc1]
= g ( [loc0]  ) & f ( [loc1] )
< g ( [loc0] ) & f ( [loc1] ) <EOS>

> find the [loc0] and then find the [loc1]
= f ( [loc0] & f ( [loc1] ) )
< f ( [loc0] & f ( [loc1] ) ) <EOS>

> stay on the [loc0] first find the [loc1] and then find the [loc2]
= g ( [loc0] ) & f ( [loc1]  f ( [loc2] ) )
< g ( [loc0] ) & f ( [loc1]  f ( [loc2] ) ) <EOS>

> do not enter [loc0] first find the [loc1] and then find the [loc2]
= g ( ! [loc0] ) & f ( [loc1]  f ( [loc2] ) )
< g ( ! [loc0] ) & f ( [loc1]  f ( [loc2] ) ) <EOS>

> go to the [loc0] and then go to the [loc1]
= f ( [loc0] & f ( [loc1] ) )
< f ( [loc0] & f ( [loc1] ) ) <EOS>

> find the [loc0] and then find the kabob and [loc1]
= f ( [loc0] & f ( kabob and [loc1] ) )
< f ( [loc0] & f ( [loc1] and [loc2] ) ) <EOS>

> do not enter [loc0] 

< f ( [loc0] & f ( [loc1] ) ) <EOS>

> find the [loc0] and then find the kabob and [loc1]
= f ( [loc0] & f ( kabob and [loc1] ) )
< f ( [loc0] & f ( [loc1] and [loc2] ) ) <EOS>

> do not enter [loc0] first find the [loc1] and then find the [loc2]
= g ( ! [loc0] ) & f ( [loc1]  f ( [loc2] ) )
< g ( ! [loc0] ) & f ( [loc1]  f ( [loc2] ) ) <EOS>

> go to the [loc0] and then go to the [loc1] and [loc2]
= f ( [loc0] & f ( [loc1] and [loc2] ) )
< f ( [loc0] & f ( [loc1] and [loc2] ) ) <EOS>

> stay away from [loc0] and find the [loc1]
= g ( ! [loc0] ) & f ( [loc1] )
< g ( ! [loc0] ) & f ( [loc1] ) <EOS>

> find the [loc0] and then find the [loc1]
= f ( [loc0] & f ( [loc1] ) )
< f ( [loc0] & f ( [loc1] ) ) <EOS>

> do not enter [loc0] first find the [loc1] and then find the [loc2]
= g ( ! [loc0] ) & f ( [loc1]  f ( [loc2] ) )
< g ( ! [loc0] ) & f ( [loc1]  f ( [loc2] ) ) <EOS>

> go to the [loc0] and then go to the [loc1] and [loc2]
= f ( [loc0] & f ( [loc1] and [loc2] ) )
< f ( [loc0] & f ( 

In [10]:
evaluateExampleWithName("go to the cvs and then go to the bookstore")

input =          go to the cvs and then go to the bookstore
masked input =  go to the cvs and then go to the [loc0]
model output = f ( [loc0] ) <EOS>
mapped output = f ( bookstore ) <EOS>


In [46]:
evaluateExampleWithName("go to the bookstore and then go to the kabob and curry")

input =          go to the bookstore and then go to the kabob and curry
masked input =  go to the [loc0] and then go to the kabob and [loc1]
model output = f ( [loc0] & f ( [loc1] and [loc2] ) ) <EOS>
mapped output = f ( bookstore & f ( curry and [loc2] ) ) <EOS>


In [11]:
evaluateExampleWithName("go to kabob and curry")

input =          go to kabob and curry
masked input =  go to [loc0] and [loc1]
model output = f ( [loc0] ) & f ( [loc1] and [loc2] ) ) <EOS>
mapped output = f ( kabob ) & f ( curry and [loc2] ) ) <EOS>


In [13]:
evaluateExampleWithName("do not enter angell st find the chipotle and then find the library")

input =          do not enter angell st find the chipotle and then find the library
masked input =  do not enter [loc0] find the [loc1] and then find the [loc2]
model output = g ( ! [loc0] ) & f ( [loc1]  f ( [loc2] ) ) <EOS>
mapped output = g ( ! angell st ) & f ( chipotle  f ( library ) ) <EOS>


In [14]:
evaluateExampleWithName("do not enter angell st find the bookstore and then find the kabob and curry")

input =          do not enter angell st find the bookstore and then find the kabob and curry
masked input =  do not enter [loc0] find the [loc1] and then find the kabob and [loc2]
model output = g ( ! [loc0] ) & f ( [loc1] and [loc2] ) ) <EOS>
mapped output = g ( ! angell st ) & f ( bookstore and curry ) ) <EOS>


In [15]:
evaluateExampleWithName("do not enter angell st find the bookstore and then find the cvs")

input =          do not enter angell st find the bookstore and then find the cvs
masked input =  do not enter [loc0] find the [loc1] and then find the cvs
model output = g ( ! [loc0] ) & f ( [loc1] and [loc2] ) ) <EOS>
mapped output = g ( ! angell st ) & f ( bookstore and [loc2] ) ) <EOS>


In [18]:
evaluateExampleWithName("go to brook st and then go to bookstore")

input =          go to brook st and then go to bookstore
masked input =  go to [loc0] and then go to [loc1]
model output = f ( [loc0] & f ( [loc1] ) ) <EOS>
mapped output = f ( brook st & f ( bookstore ) ) <EOS>


In [19]:
evaluateExampleWithName("go to angell st and go to bookstore")

input =          go to angell st and go to bookstore
masked input =  go to [loc0] and go to [loc1]
model output = f ( [loc0] & f ( [loc1] ) ) <EOS>
mapped output = f ( angell st & f ( bookstore ) ) <EOS>


In [20]:
evaluateExampleWithName("go to starbucks and go to bookstore")

input =          go to starbucks and go to bookstore
masked input =  go to [loc0] and go to [loc1]
model output = f ( [loc0] & f ( [loc1] ) ) <EOS>
mapped output = f ( starbucks & f ( bookstore ) ) <EOS>


In [21]:
evaluateExampleWithName("go to thayer st and go to bookstore")

input =          go to thayer st and go to bookstore
masked input =  go to [loc0] and go to [loc1]
model output = f ( [loc0] & f ( [loc1] ) ) <EOS>
mapped output = f ( thayer st & f ( bookstore ) ) <EOS>


In [22]:
evaluateExampleWithName("stay on the thayer st and go to bookstore")

input =          stay on the thayer st and go to bookstore
masked input =  stay on the [loc0] and go to [loc1]
model output = f ( [loc0] & f ( [loc1] ) ) <EOS>
mapped output = f ( thayer st & f ( bookstore ) ) <EOS>


In [23]:
evaluateExampleWithName("stay on the starbucks and go to bookstore")

input =          stay on the starbucks and go to bookstore
masked input =  stay on the [loc0] and go to [loc1]
model output = f ( [loc0] & f ( [loc1] ) ) <EOS>
mapped output = f ( starbucks & f ( bookstore ) ) <EOS>


In [43]:
evaluateExampleWithName("stay on the starbucks and go to thayer st")

input =          stay on the starbucks and go to thayer st
masked input =  stay on the [loc0] and go to [loc1]
model output = g ( [loc0] ) & f ( [loc1] ) <EOS>
mapped output = g ( starbucks ) & f ( thayer st ) <EOS>


In [44]:
evaluateExampleWithName("stay on the citizen's bank and find the bookstore")

input =          stay on the citizen's bank and find the bookstore
masked input =  stay on the [loc0] and find the [loc1]
model output = g ( [loc0] ) & f ( [loc1] ) <EOS>
mapped output = g ( citizen's bank ) & f ( bookstore ) <EOS>


In [45]:
evaluateExampleWithName("stay on the thayer st and find the brook st")

input =          stay on the thayer st and find the brook st
masked input =  stay on the [loc0] and find the [loc1]
model output = g ( [loc0] ) & f ( [loc1] ) <EOS>
mapped output = g ( thayer st ) & f ( brook st ) <EOS>


In [26]:
evaluateExampleWithName("first go to starbucks and then go to bookstore")

input =          first go to starbucks and then go to bookstore
masked input =  first go to [loc0] and then go to [loc1]
model output = f ( [loc0] & f ( [loc1] ) ) <EOS>
mapped output = f ( starbucks & f ( bookstore ) ) <EOS>


In [27]:
evaluateExampleWithName("find the barus building and then find the cvs")

input =          find the barus building and then find the cvs
masked input =  find the [loc0] and then find the cvs
model output = f ( [loc0] & f ( [loc1] ) ) <EOS>
mapped output = f ( barus building & f ( [loc1] ) ) <EOS>


In [28]:
evaluateExampleWithName("find the bookstore and then find the nelson fitness center")

input =          find the bookstore and then find the nelson fitness center
masked input =  find the [loc0] and then find the [loc1]
model output = f ( [loc0] & f ( [loc1] ) ) <EOS>
mapped output = f ( bookstore & f ( nelson fitness center ) ) <EOS>


In [29]:
evaluateExampleWithName("stay on angell st first find the bookstore and then find the cvs")

input =          stay on angell st first find the bookstore and then find the cvs
masked input =  stay on [loc0] first find the [loc1] and then find the cvs
model output = g ( ! [loc0] ) & f ( [loc1] and [loc2] ) ) <EOS>
mapped output = g ( ! angell st ) & f ( bookstore and [loc2] ) ) <EOS>


In [35]:
evaluateExampleWithName("go to the starbucks")

input =          go to the starbucks
masked input =  go to the [loc0]
model output = f ( [loc0] ) <EOS>
mapped output = f ( starbucks ) <EOS>


In [48]:
evaluateExampleWithName("do not enter the angell st find the starbucks and then find the bookstore")

input =          do not enter the angell st find the starbucks and then find the bookstore
masked input =  do not enter the [loc0] find the [loc1] and then find the [loc2]
model output = g ( ! [loc0] ) & f ( [loc1]  f ( [loc2] ) ) <EOS>
mapped output = g ( ! angell st ) & f ( starbucks  f ( bookstore ) ) <EOS>


https://www.comet.ml/suchzheng2/lang2ltl/8e359a23a1d04e26909ef96bfa42c6da?experiment-tab=chart&showOutliers=true&smoothing=0&transformY=smoothing&xAxis=step

In [30]:
text = "do not enter Meeting St  find the bookstore and then find the CVS"
doc = location_sniffer(text)
print(text)
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

do not enter Meeting St  find the bookstore and then find the CVS
Entities [('Meeting St', 'LOC'), ('bookstore', 'LOC'), ('CVS', 'LOC')]


In [31]:
text = "do not enter thayer st find the bookstore and then find the cvs"
doc = location_sniffer(text)
print(text)
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

do not enter thayer st find the bookstore and then find the cvs
Entities [('thayer st', 'LOC'), ('bookstore', 'LOC')]


In [32]:
text = "stay on angell st find the bookstore"
doc = location_sniffer(text)
print(text)
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

stay on angell st find the bookstore
Entities [('angell st', 'LOC'), ('bookstore', 'LOC')]


In [33]:
text = "stay on brook st find the bookstore"
doc = location_sniffer(text)
print(text)
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

stay on brook st find the bookstore
Entities [('brook st', 'LOC'), ('bookstore', 'LOC')]
